In [ ]:
# ============================================================================
# BLW FACE RECOGNITION ATTENDANCE SYSTEM - BACKEND WITH DELETE
# Banaras Locomotive Works - Department of Computer Science & Engineering
# ============================================================================

import cv2
import numpy as np
import torch
from facenet_pytorch import InceptionResnetV1
from scipy.spatial.distance import cosine
import sqlite3
from datetime import datetime, date
import pickle
import warnings

warnings.filterwarnings('ignore')

print("="*80)
print("🏭 BLW FACE RECOGNITION ATTENDANCE SYSTEM - BACKEND")
print("="*80)

# ============================================================================
# DATABASE CLASS WITH DELETE EMPLOYEE
# ============================================================================

class AttendanceDatabase:
    """SQLite database for employee records and attendance"""
    
    def __init__(self, db_path='blw_attendance.db'):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.cursor = self.conn.cursor()
        self._create_tables()
        print("✅ Database initialized")
    
    def _create_tables(self):
        """Create necessary tables"""
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS employees (
                emp_id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                department TEXT,
                designation TEXT,
                registration_date TEXT
            )
        ''')
        
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS face_embeddings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                emp_id TEXT,
                embedding BLOB,
                FOREIGN KEY (emp_id) REFERENCES employees(emp_id)
            )
        ''')
        
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS attendance (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                emp_id TEXT,
                date TEXT,
                check_in_time TEXT,
                confidence REAL,
                FOREIGN KEY (emp_id) REFERENCES employees(emp_id)
            )
        ''')
        
        # Admin password table
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS admin_config (
                key TEXT PRIMARY KEY,
                value TEXT
            )
        ''')
        
        # Set default password if not exists
        self.cursor.execute("SELECT value FROM admin_config WHERE key='password'")
        if not self.cursor.fetchone():
            self.cursor.execute("INSERT INTO admin_config VALUES ('password', 'admin123')")
        
        self.conn.commit()
    
    def register_employee(self, emp_id, name, department, designation):
        """Register new employee"""
        try:
            self.cursor.execute('''
                INSERT INTO employees VALUES (?, ?, ?, ?, ?)
            ''', (emp_id, name, department, designation, datetime.now().strftime('%Y-%m-%d')))
            self.conn.commit()
            return True
        except sqlite3.IntegrityError:
            return False
    
    def delete_employee(self, emp_id):
        """Delete employee and all related data (face embeddings & attendance)"""
        try:
            # Delete face embeddings
            self.cursor.execute('DELETE FROM face_embeddings WHERE emp_id = ?', (emp_id,))
            
            # Delete attendance records
            self.cursor.execute('DELETE FROM attendance WHERE emp_id = ?', (emp_id,))
            
            # Delete employee
            self.cursor.execute('DELETE FROM employees WHERE emp_id = ?', (emp_id,))
            
            self.conn.commit()
            return True
        except Exception as e:
            print(f"Error deleting employee: {e}")
            return False
    
    def save_face_embedding(self, emp_id, embedding):
        """Save face embedding for employee"""
        embedding_blob = pickle.dumps(embedding)
        self.cursor.execute('''
            INSERT INTO face_embeddings (emp_id, embedding) VALUES (?, ?)
        ''', (emp_id, embedding_blob))
        self.conn.commit()
    
    def get_all_embeddings(self):
        """Get all face embeddings"""
        self.cursor.execute('SELECT emp_id, embedding FROM face_embeddings')
        results = []
        for emp_id, embedding_blob in self.cursor.fetchall():
            embedding = pickle.loads(embedding_blob)
            results.append((emp_id, embedding))
        return results
    
    def mark_attendance(self, emp_id, confidence):
        """Mark attendance for employee"""
        today = date.today().strftime('%Y-%m-%d')
        now = datetime.now().strftime('%H:%M:%S')
        
        self.cursor.execute('''
            SELECT * FROM attendance WHERE emp_id = ? AND date = ?
        ''', (emp_id, today))
        
        if self.cursor.fetchone():
            return False
        
        self.cursor.execute('''
            INSERT INTO attendance (emp_id, date, check_in_time, confidence)
            VALUES (?, ?, ?, ?)
        ''', (emp_id, today, now, confidence))
        self.conn.commit()
        return True
    
    def get_employee_info(self, emp_id):
        """Get employee information"""
        self.cursor.execute('''
            SELECT name, department, designation FROM employees WHERE emp_id = ?
        ''', (emp_id,))
        result = self.cursor.fetchone()
        if result:
            return {'name': result[0], 'department': result[1], 'designation': result[2]}
        return None
    
    def get_today_attendance(self):
        """Get today's attendance records"""
        today = date.today().strftime('%Y-%m-%d')
        self.cursor.execute('''
            SELECT a.id, e.emp_id, e.name, e.department, a.check_in_time, a.confidence, a.date
            FROM attendance a
            JOIN employees e ON a.emp_id = e.emp_id
            WHERE a.date = ?
            ORDER BY a.check_in_time
        ''', (today,))
        return self.cursor.fetchall()
    
    def get_all_attendance(self):
        """Get all attendance records"""
        self.cursor.execute('''
            SELECT a.id, e.emp_id, e.name, e.department, a.check_in_time, a.confidence, a.date
            FROM attendance a
            JOIN employees e ON a.emp_id = e.emp_id
            ORDER BY a.date DESC, a.check_in_time DESC
        ''')
        return self.cursor.fetchall()
    
    def delete_attendance(self, attendance_id):
        """Delete attendance record"""
        self.cursor.execute('DELETE FROM attendance WHERE id = ?', (attendance_id,))
        self.conn.commit()
    
    def get_all_employees(self):
        """Get all registered employees"""
        self.cursor.execute('SELECT emp_id, name, department, designation, registration_date FROM employees')
        return self.cursor.fetchall()
    
    def get_employee_face_count(self, emp_id):
        """Get number of face embeddings for an employee"""
        self.cursor.execute('SELECT COUNT(*) FROM face_embeddings WHERE emp_id = ?', (emp_id,))
        return self.cursor.fetchone()[0]
    
    def verify_password(self, password):
        """Verify admin password"""
        self.cursor.execute("SELECT value FROM admin_config WHERE key='password'")
        result = self.cursor.fetchone()
        return result and result[0] == password
    
    def close(self):
        """Close database connection"""
        self.conn.close()

# ============================================================================
# FACE RECOGNITION CLASS
# ============================================================================

class FaceRecognitionSystem:
    """Haar Cascade + FaceNet face recognition with blur detection"""
    
    def __init__(self, threshold=0.6):
        self.np = np
        self.threshold = threshold
        
        # Haar Cascade detector
        cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        self.face_detector = cv2.CascadeClassifier(cascade_path)
        
        if self.face_detector.empty():
            raise Exception("Failed to load Haar Cascade")
        
        # FaceNet model
        self.device = torch.device('cpu')
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)
        
        print("✅ Face Recognition System initialized")
    
    def detect_faces(self, frame):
        """Detect faces in frame"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        
        faces_rect = self.face_detector.detectMultiScale(
            gray, scaleFactor=1.05, minNeighbors=3, minSize=(30, 30)
        )
        
        faces = []
        for (x, y, w, h) in faces_rect:
            faces.append((x, y, x + w, y + h))
        return faces
    
    def calculate_blur(self, image):
        """Calculate blur score using Laplacian variance"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        return laplacian_var
    
    def is_image_clear(self, frame, box, blur_threshold=100):
        """Check if face region is clear (not blurry)"""
        x1, y1, x2, y2 = box
        face_region = frame[y1:y2, x1:x2]
        
        if face_region.size == 0:
            return False
        
        blur_score = self.calculate_blur(face_region)
        return blur_score > blur_threshold
    
    def extract_face(self, frame, box, margin=20):
        """Extract face region and convert to tensor"""
        x1, y1, x2, y2 = box
        h, w = frame.shape[:2]
        
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(w, x2 + margin)
        y2 = min(h, y2 + margin)
        
        face = frame[y1:y2, x1:x2]
        if face.size == 0:
            return None
        
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_resized = cv2.resize(face_rgb, (160, 160))
        
        face_np = self.np.array(face_resized, dtype=self.np.float32)
        face_tensor = torch.from_numpy(face_np).permute(2, 0, 1)
        face_tensor = (face_tensor - 127.5) / 128.0
        face_tensor = face_tensor.unsqueeze(0).to(self.device)
        
        return face_tensor
    
    def get_embedding(self, face_tensor):
        """Generate 512D embedding from face tensor"""
        with torch.no_grad():
            embedding = self.facenet(face_tensor)
            embedding = embedding.cpu().numpy().flatten()
            embedding = embedding / self.np.linalg.norm(embedding)
        return embedding
    
    def compare_embeddings(self, embedding1, embedding2):
        """Compare two embeddings using cosine similarity"""
        distance = cosine(embedding1, embedding2)
        return 1 - distance
    
    def recognize_face(self, frame, box, known_embeddings):
        """Recognize face from known embeddings"""
        face_tensor = self.extract_face(frame, box)
        if face_tensor is None:
            return None, None
        
        face_embedding = self.get_embedding(face_tensor)
        
        best_match = None
        best_similarity = 0.0
        
        for emp_id, known_embedding in known_embeddings:
            similarity = self.compare_embeddings(face_embedding, known_embedding)
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = emp_id
        
        if best_similarity >= (1 - self.threshold):
            return best_match, best_similarity
        return None, None
    
    def register_face(self, frame, box):
        """Register new face and return embedding"""
        face_tensor = self.extract_face(frame, box)
        if face_tensor is None:
            return None
        return self.get_embedding(face_tensor)

# ============================================================================
# INITIALIZE BACKEND
# ============================================================================

db = AttendanceDatabase()
face_system = FaceRecognitionSystem(threshold=0.6)

print("="*80)
print("✅ BACKEND INITIALIZATION COMPLETE")
print(f"   Database: {db.db_path}")
print(f"   Registered Employees: {len(db.get_all_employees())}")
print(f"   Face Recognition: Ready")
print("="*80)


🏭 BLW FACE RECOGNITION ATTENDANCE SYSTEM - BACKEND
✅ Database initialized
✅ Face Recognition System initialized
✅ BACKEND INITIALIZATION COMPLETE
   Database: blw_attendance.db
   Registered Employees: 3
   Face Recognition: Ready


In [ ]:
# ============================================================================
# BLW FACE RECOGNITION ATTENDANCE SYSTEM - COMPLETE WORKING GUI
# ============================================================================

import sys
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                             QHBoxLayout, QLabel, QPushButton, QLineEdit, QTextEdit,
                             QTabWidget, QTableWidget, QTableWidgetItem, QGroupBox,
                             QFormLayout, QMessageBox, QHeaderView, QListWidget,
                             QDialog, QDialogButtonBox, QInputDialog, QGridLayout)
from PyQt5.QtCore import Qt, QTimer, pyqtSignal, QThread, QSize
from PyQt5.QtGui import QImage, QPixmap, QFont
import cv2
import numpy as np
from datetime import datetime
import time

print("="*80)
print("🖥️  INITIALIZING COMPLETE WORKING GUI")
print("="*80)

# ============================================================================
# VIDEO THREAD
# ============================================================================

class VideoThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)
    
    def __init__(self, fps=15):
        super().__init__()
        self._run_flag = True
        self.fps = fps
        self.delay = 1.0 / fps
    
    def run(self):
        cap = cv2.VideoCapture(0)
        while self._run_flag:
            ret, frame = cap.read()
            if ret:
                self.change_pixmap_signal.emit(frame)
            time.sleep(self.delay)
        cap.release()
    
    def stop(self):
        self._run_flag = False
        self.wait()

# ============================================================================
# SIMPLIFIED RELIABLE AUTO-CAPTURE THREAD
# ============================================================================

class AutoCaptureThread(QThread):
    frame_signal = pyqtSignal(np.ndarray, str)
    capture_complete = pyqtSignal(list, list)
    thumbnail_captured = pyqtSignal(np.ndarray, int)
    
    def __init__(self, blur_threshold=30):
        super().__init__()
        self._run_flag = True
        self.blur_threshold = blur_threshold
        self.captured_embeddings = []
        self.captured_images = []
        self.last_capture_time = 0
        self.capture_delay = 1.0
    
    def run(self):
        cap = cv2.VideoCapture(0)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        while self._run_flag and len(self.captured_embeddings) < 5:
            ret, frame = cap.read()
            if not ret:
                continue
            
            faces = face_system.detect_faces(frame)
            display_frame = frame.copy()
            current_time = time.time()
            
            if faces:
                x1, y1, x2, y2 = faces[0]
                
                face_width = x2 - x1
                face_height = y2 - y1
                
                is_good_size = face_width >= 80 and face_height >= 80
                time_ok = (current_time - self.last_capture_time) > self.capture_delay
                
                margin = 10
                is_in_frame = (
                    x1 > margin and 
                    y1 > margin and 
                    x2 < (frame_width - margin) and 
                    y2 < (frame_height - margin)
                )
                
                if is_good_size and time_ok and is_in_frame:
                    embedding = face_system.register_face(frame, faces[0])
                    
                    if embedding is not None:
                        self.captured_embeddings.append(embedding)
                        
                        face_img = frame[y1:y2, x1:x2].copy()
                        self.captured_images.append(face_img)
                        
                        self.thumbnail_captured.emit(face_img, len(self.captured_embeddings) - 1)
                        
                        self.last_capture_time = current_time
                        
                        cv2.rectangle(display_frame, (0, 0), (frame_width, frame_height), (0, 255, 0), 15)
                        cv2.rectangle(display_frame, (x1-10, y1-10), (x2+10, y2+10), (0, 255, 0), 5)
                        cv2.putText(display_frame, f"✓ CAPTURED {len(self.captured_embeddings)}/5", 
                                  (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)
                        status = f"✅ Captured {len(self.captured_embeddings)}/5"
                        
                        self.frame_signal.emit(display_frame, status)
                        time.sleep(0.3)
                        continue
                else:
                    if not is_good_size:
                        color = (0, 0, 255)
                        text = "Come closer to camera"
                    elif not is_in_frame:
                        color = (255, 0, 255)
                        text = "Center your face"
                    else:
                        color = (0, 255, 0)
                        text = f"Ready... {int(self.capture_delay - (current_time - self.last_capture_time))}s"
                    
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 3)
                    cv2.putText(display_frame, text, (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                    status = f"📸 {len(self.captured_embeddings)}/5 - {text}"
            else:
                status = f"📸 {len(self.captured_embeddings)}/5 - Show your face"
                
                center_x = frame_width // 2
                center_y = frame_height // 2
                box_size = 200
                cv2.rectangle(display_frame, 
                            (center_x - box_size, center_y - box_size), 
                            (center_x + box_size, center_y + box_size), 
                            (255, 255, 255), 2)
                cv2.putText(display_frame, "Position face here", 
                          (center_x - 150, center_y - box_size - 20), 
                          cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
            
            self.frame_signal.emit(display_frame, status)
            time.sleep(0.04)
        
        cap.release()
        self.capture_complete.emit(self.captured_embeddings, self.captured_images)
    
    def stop(self):
        self._run_flag = False
        self.wait()

# ============================================================================
# ATTENDANCE THREAD
# ============================================================================

class AttendanceThread(QThread):
    frame_signal = pyqtSignal(np.ndarray)
    attendance_marked = pyqtSignal(str, str, float)
    
    def __init__(self):
        super().__init__()
        self._run_flag = True
        self.known_embeddings = []
        self.recent_recognitions = {}
        self.recognition_cooldown = 5
        self.frame_count = 0
    
    def run(self):
        cap = cv2.VideoCapture(0)
        
        while self._run_flag:
            ret, frame = cap.read()
            if not ret:
                continue
            
            self.frame_count += 1
            
            if self.frame_count % 5 == 0:
                faces = face_system.detect_faces(frame)
                current_time = datetime.now()
                display_frame = frame.copy()
                
                for box in faces:
                    x1, y1, x2, y2 = box
                    emp_id, confidence = face_system.recognize_face(frame, box, self.known_embeddings)
                    
                    if emp_id:
                        if emp_id in self.recent_recognitions:
                            time_diff = (current_time - self.recent_recognitions[emp_id]).total_seconds()
                            if time_diff < self.recognition_cooldown:
                                cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                emp_info = db.get_employee_info(emp_id)
                                cv2.putText(display_frame, emp_info['name'], (x1, y1-10), 
                                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                                continue
                        
                        marked = db.mark_attendance(emp_id, confidence)
                        
                        if marked:
                            self.recent_recognitions[emp_id] = current_time
                            emp_info = db.get_employee_info(emp_id)
                            
                            cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
                            cv2.putText(display_frame, emp_info['name'], (x1, y1-40), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                            cv2.putText(display_frame, f"{confidence:.1%}", (x1, y1-15), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                            cv2.putText(display_frame, "MARKED", (x1, y2+25), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                            
                            self.attendance_marked.emit(emp_info['name'], emp_id, confidence)
                        else:
                            emp_info = db.get_employee_info(emp_id)
                            cv2.rectangle(display_frame, (x1, y1), (x2, y2), (255, 165, 0), 2)
                            cv2.putText(display_frame, emp_info['name'], (x1, y1-40), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 165, 0), 2)
                            cv2.putText(display_frame, "Already Marked Today", (x1, y1-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2)
                    else:
                        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                        cv2.putText(display_frame, "UNKNOWN", (x1, y1-10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                
                timestamp = datetime.now().strftime('%H:%M:%S')
                cv2.rectangle(display_frame, (5, 5), (350, 90), (0, 0, 0), -1)
                cv2.putText(display_frame, f"LIVE - {timestamp}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.putText(display_frame, f"Today: {len(self.recent_recognitions)} marked", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                self.frame_signal.emit(display_frame)
            
            time.sleep(0.02)
        
        cap.release()
    
    def stop(self):
        self._run_flag = False
        self.wait()

# ============================================================================
# PASSWORD DIALOG
# ============================================================================

class PasswordDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Admin Authentication")
        self.setModal(True)
        
        layout = QVBoxLayout()
        
        layout.addWidget(QLabel("🔒 Enter Admin Password:"))
        
        self.password_input = QLineEdit()
        self.password_input.setEchoMode(QLineEdit.Password)
        self.password_input.setPlaceholderText("Default: admin123")
        layout.addWidget(self.password_input)
        
        buttons = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        buttons.accepted.connect(self.accept)
        buttons.rejected.connect(self.reject)
        layout.addWidget(buttons)
        
        self.setLayout(layout)
    
    def get_password(self):
        return self.password_input.text()

# ============================================================================
# MAIN WINDOW
# ============================================================================

class BLWAttendanceSystem(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("🏭 BLW Face Recognition Attendance System")
        self.setGeometry(100, 100, 1500, 900)
        
        self.setStyleSheet("""
            QMainWindow { background-color: #f5f5f5; }
            QTabWidget::pane { border: 2px solid #4CAF50; border-radius: 5px; background: white; }
            QTabBar::tab {
                background: #e0e0e0; padding: 10px 20px; margin: 2px;
                border-radius: 5px; font-size: 14px; font-weight: bold; color: black;
            }
            QTabBar::tab:selected { background: #4CAF50; color: white; }
            QPushButton {
                background-color: #4CAF50; color: black; border: none;
                padding: 10px 20px; font-size: 14px; font-weight: bold; border-radius: 5px;
            }
            QPushButton:hover { background-color: #45a049; }
            QPushButton:disabled { background-color: #cccccc; color: #666666; }
            QLineEdit {
                padding: 8px; border: 2px solid #ddd; border-radius: 5px; 
                font-size: 13px; background: white; color: black;
            }
            QLineEdit:focus { border: 2px solid #4CAF50; }
            QLabel { color: #333333; }
            QGroupBox { 
                background-color: white; border: 2px solid #ddd; 
                border-radius: 8px; padding: 15px; margin: 5px;
                font-weight: bold; font-size: 14px;
            }
            QGroupBox::title { 
                color: #333333; font-weight: bold; 
                subcontrol-origin: margin; subcontrol-position: top left;
                padding: 5px 10px;
            }
            QWidget { background-color: white; }
            QTableWidget {
                background-color: white;
                color: black;
                gridline-color: #ddd;
                selection-background-color: #4CAF50;
                selection-color: white;
            }
            QTableWidget::item {
                color: black;
                padding: 5px;
            }
            QHeaderView::section {
                background-color: #4CAF50;
                color: white;
                padding: 8px;
                border: 1px solid #ddd;
                font-weight: bold;
            }
        """)
        
        self.video_thread = None
        self.auto_capture_thread = None
        self.attendance_thread = None
        self.captured_embeddings = []
        self.captured_images = []
        self.thumbnail_labels = []
        
        self.tabs = QTabWidget()
        self.tabs.currentChanged.connect(self.on_tab_changed)
        self.setCentralWidget(self.tabs)
        
        self.create_registration_tab()
        self.create_attendance_tab()
        self.create_reports_tab()
        self.create_admin_tab()
        
        print("✅ GUI Initialized")
    
    def on_tab_changed(self, index):
        if hasattr(self, 'admin_lock_widget') and hasattr(self, 'admin_content_widget'):
            self.admin_lock_widget.show()
            self.admin_content_widget.hide()
    
    # ========================================================================
    # REGISTRATION TAB
    # ========================================================================
    
    def create_registration_tab(self):
        tab = QWidget()
        main_layout = QVBoxLayout()
        main_layout.setSpacing(15)
        main_layout.setContentsMargins(20, 20, 20, 20)
        
        top_layout = QHBoxLayout()
        top_layout.setSpacing(20)
        
        form_group = QGroupBox("Employee Information")
        form_group.setMinimumWidth(400)
        form_layout = QFormLayout()
        form_layout.setSpacing(15)
        form_layout.setContentsMargins(20, 20, 20, 20)
        form_layout.setLabelAlignment(Qt.AlignRight)
        
        self.reg_id = QLineEdit()
        self.reg_id.setPlaceholderText("BLW001")
        self.reg_id.setMinimumHeight(40)
        
        self.reg_name = QLineEdit()
        self.reg_name.setPlaceholderText("Full Name")
        self.reg_name.setMinimumHeight(40)
        
        self.reg_dept = QLineEdit()
        self.reg_dept.setPlaceholderText("Production")
        self.reg_dept.setMinimumHeight(40)
        
        self.reg_desig = QLineEdit()
        self.reg_desig.setPlaceholderText("Engineer")
        self.reg_desig.setMinimumHeight(40)
        
        form_layout.addRow("Employee ID:", self.reg_id)
        form_layout.addRow("Name:", self.reg_name)
        form_layout.addRow("Department:", self.reg_dept)
        form_layout.addRow("Designation:", self.reg_desig)
        
        self.reg_status = QLabel("📸 Ready")
        self.reg_status.setStyleSheet("color: #666; font-size: 14px; font-weight: bold; margin-top: 10px;")
        form_layout.addRow("", self.reg_status)
        
        self.reg_capture_btn = QPushButton("📷 Start Auto-Capture")
        self.reg_capture_btn.setMinimumHeight(45)
        self.reg_capture_btn.clicked.connect(self.start_auto_capture)
        
        self.reg_register_btn = QPushButton("✅ Register")
        self.reg_register_btn.setMinimumHeight(45)
        self.reg_register_btn.clicked.connect(self.register_employee)
        self.reg_register_btn.setEnabled(False)
        
        self.reg_recapture_btn = QPushButton("🔄 Re-Capture All")
        self.reg_recapture_btn.setMinimumHeight(45)
        self.reg_recapture_btn.setStyleSheet("background-color: #ff9800; color: black;")
        self.reg_recapture_btn.clicked.connect(self.recapture_images)
        self.reg_recapture_btn.setEnabled(False)
        
        form_layout.addRow("", self.reg_capture_btn)
        form_layout.addRow("", self.reg_recapture_btn)
        form_layout.addRow("", self.reg_register_btn)
        
        info_label = QLabel("ℹ️ Auto-capture: 5 clear images (1s interval).\nJust show your face!")
        info_label.setStyleSheet("color: #666; font-size: 11px; margin-top: 10px;")
        info_label.setWordWrap(True)
        form_layout.addRow("", info_label)
        
        form_group.setLayout(form_layout)
        
        preview_group = QGroupBox("Camera Preview")
        preview_group.setMinimumWidth(650)
        preview_layout = QVBoxLayout()
        preview_layout.setContentsMargins(15, 15, 15, 15)
        
        self.reg_camera_label = QLabel()
        self.reg_camera_label.setMinimumSize(640, 480)
        self.reg_camera_label.setStyleSheet("border: 2px solid #4CAF50; background: #000;")
        self.reg_camera_label.setAlignment(Qt.AlignCenter)
        self.reg_camera_label.setText("Camera Preview")
        
        preview_layout.addWidget(self.reg_camera_label)
        preview_group.setLayout(preview_layout)
        
        top_layout.addWidget(form_group, 2)
        top_layout.addWidget(preview_group, 3)
        
        thumb_group = QGroupBox("Captured Images (5)")
        thumb_layout = QHBoxLayout()
        thumb_layout.setSpacing(10)
        
        self.thumbnail_labels = []
        for i in range(5):
            thumb_label = QLabel()
            thumb_label.setFixedSize(120, 120)
            thumb_label.setStyleSheet("border: 2px solid #ddd; background: #f0f0f0;")
            thumb_label.setAlignment(Qt.AlignCenter)
            thumb_label.setText(f"{i+1}")
            self.thumbnail_labels.append(thumb_label)
            thumb_layout.addWidget(thumb_label)
        
        thumb_group.setLayout(thumb_layout)
        
        main_layout.addLayout(top_layout)
        main_layout.addWidget(thumb_group)
        
        tab.setLayout(main_layout)
        self.tabs.addTab(tab, "📝 Registration")
    
    def start_auto_capture(self):
        if not self.reg_id.text() or not self.reg_name.text():
            QMessageBox.warning(self, "Warning", "Enter ID and Name!")
            return
        
        for label in self.thumbnail_labels:
            label.clear()
            label.setText("⏳")
        
        self.captured_embeddings = []
        self.captured_images = []
        self.reg_capture_btn.setEnabled(False)
        self.reg_recapture_btn.setEnabled(False)
        self.reg_status.setText("📸 Auto-capturing...")
        
        self.auto_capture_thread = AutoCaptureThread(blur_threshold=30)
        self.auto_capture_thread.frame_signal.connect(self.update_auto_capture_frame)
        self.auto_capture_thread.thumbnail_captured.connect(self.update_thumbnail)
        self.auto_capture_thread.capture_complete.connect(self.on_auto_capture_complete)
        self.auto_capture_thread.start()
    
    def update_thumbnail(self, image, index):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb.shape
        qt_image = QImage(rgb.data, w, h, ch * w, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image).scaled(120, 120, Qt.KeepAspectRatio)
        self.thumbnail_labels[index].setPixmap(pixmap)
    
    def update_auto_capture_frame(self, frame, status):
        self.reg_status.setText(status)
        
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        qt_image = QImage(rgb_image.data, w, h, ch * w, QImage.Format_RGB888)
        self.reg_camera_label.setPixmap(QPixmap.fromImage(qt_image).scaled(
            640, 480, Qt.KeepAspectRatio))
    
    def on_auto_capture_complete(self, embeddings, images):
        self.captured_embeddings = embeddings
        self.captured_images = images
        self.reg_capture_btn.setEnabled(True)
        
        if len(embeddings) >= 5:
            self.reg_status.setText(f"✅ Captured {len(embeddings)}/5 - Ready!")
            self.reg_register_btn.setEnabled(True)
            self.reg_recapture_btn.setEnabled(True)
            QMessageBox.information(self, "Success", 
                f"✅ Captured {len(embeddings)} clear images!\n\nClick Register to save.")
        else:
            self.reg_status.setText(f"⚠️ Only {len(embeddings)}/5 - Try again")
            self.reg_recapture_btn.setEnabled(True)
            QMessageBox.warning(self, "Incomplete", 
                f"Only captured {len(embeddings)} images.\n\nClick Re-Capture to try again.")
    
    def recapture_images(self):
        self.start_auto_capture()
    
    def register_employee(self):
        emp_id = self.reg_id.text()
        name = self.reg_name.text()
        dept = self.reg_dept.text()
        desig = self.reg_desig.text()
        
        if len(self.captured_embeddings) < 3:
            QMessageBox.warning(self, "Warning", "Need 3+ images!")
            return
        
        if db.register_employee(emp_id, name, dept, desig):
            for emb in self.captured_embeddings:
                db.save_face_embedding(emp_id, emb)
            
            QMessageBox.information(self, "Success", 
                f"✅ Registered!\n\n{name}\n{emp_id}\n{len(self.captured_embeddings)} images")
            
            self.reg_id.clear()
            self.reg_name.clear()
            self.reg_dept.clear()
            self.reg_desig.clear()
            self.captured_embeddings = []
            self.captured_images = []
            self.reg_register_btn.setEnabled(False)
            self.reg_recapture_btn.setEnabled(False)
            self.reg_status.setText("📸 Ready")
            self.reg_camera_label.setText("Camera Preview")
            
            for i, label in enumerate(self.thumbnail_labels):
                label.clear()
                label.setText(f"{i+1}")
        else:
            QMessageBox.warning(self, "Error", f"ID '{emp_id}' exists!")
    
    # ========================================================================
    # ATTENDANCE TAB
    # ========================================================================
    
    def create_attendance_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()
        
        self.att_camera_label = QLabel()
        self.att_camera_label.setMinimumSize(1200, 650)
        self.att_camera_label.setStyleSheet("border: 3px solid #f44336; background: #000;")
        self.att_camera_label.setAlignment(Qt.AlignCenter)
        self.att_camera_label.setText("Click START to begin monitoring")
        
        self.att_log = QListWidget()
        self.att_log.setMaximumHeight(150)
        
        control_layout = QHBoxLayout()
        
        self.att_start_btn = QPushButton("▶️ Start Monitoring")
        self.att_start_btn.setStyleSheet("background-color: #4CAF50; color: black; font-size: 16px; padding: 15px;")
        self.att_start_btn.clicked.connect(self.start_attendance_monitoring)
        
        self.att_stop_btn = QPushButton("⏹️ Stop")
        self.att_stop_btn.setStyleSheet("background-color: #f44336; color: black; font-size: 16px; padding: 15px;")
        self.att_stop_btn.clicked.connect(self.stop_attendance_monitoring)
        self.att_stop_btn.setEnabled(False)
        
        self.att_status = QLabel("⚪ Ready")
        self.att_status.setStyleSheet("font-size: 16px; font-weight: bold; color: #333;")
        
        control_layout.addWidget(self.att_start_btn)
        control_layout.addWidget(self.att_stop_btn)
        control_layout.addWidget(self.att_status)
        control_layout.addStretch()
        
        layout.addWidget(QLabel("🎥 Live Camera Feed"))
        layout.addWidget(self.att_camera_label)
        layout.addWidget(QLabel("📋 Attendance Log"))
        layout.addWidget(self.att_log)
        layout.addLayout(control_layout)
        
        tab.setLayout(layout)
        self.tabs.addTab(tab, "📹 Live Attendance")
    
    def start_attendance_monitoring(self):
        self.att_start_btn.setEnabled(False)
        self.att_stop_btn.setEnabled(True)
        self.att_status.setText("🔴 LIVE")
        self.att_log.clear()
        
        self.attendance_thread = AttendanceThread()
        self.attendance_thread.known_embeddings = db.get_all_embeddings()
        self.attendance_thread.frame_signal.connect(self.update_attendance_frame)
        self.attendance_thread.attendance_marked.connect(self.on_attendance_marked)
        self.attendance_thread.start()
    
    def update_attendance_frame(self, frame):
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        qt_image = QImage(rgb_image.data, w, h, ch * w, QImage.Format_RGB888)
        self.att_camera_label.setPixmap(QPixmap.fromImage(qt_image).scaled(
            1200, 650, Qt.KeepAspectRatio))
    
    def on_attendance_marked(self, name, emp_id, confidence):
        timestamp = datetime.now().strftime('%H:%M:%S')
        log_entry = f"✅ {timestamp} - {name} ({emp_id}) - {confidence:.1%}"
        self.att_log.insertItem(0, log_entry)
    
    def stop_attendance_monitoring(self):
        if self.attendance_thread:
            self.attendance_thread.stop()
        
        self.att_start_btn.setEnabled(True)
        self.att_stop_btn.setEnabled(False)
        self.att_status.setText("⚪ Stopped")
        self.att_camera_label.clear()
        self.att_camera_label.setText("Monitoring stopped - Click START to resume")
    
    # ========================================================================
    # REPORTS TAB
    # ========================================================================
    
    def create_reports_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()
        
        control_layout = QHBoxLayout()
        
        refresh_btn = QPushButton("🔄 Refresh")
        refresh_btn.clicked.connect(self.load_attendance_report)
        
        self.delete_btn = QPushButton("🗑️ Delete Selected")
        self.delete_btn.setStyleSheet("background-color: #f44336; color: black;")
        self.delete_btn.clicked.connect(self.delete_attendance)
        
        control_layout.addWidget(refresh_btn)
        control_layout.addWidget(self.delete_btn)
        control_layout.addStretch()
        
        self.report_table = QTableWidget()
        self.report_table.setColumnCount(7)
        self.report_table.setHorizontalHeaderLabels(["ID", "Emp ID", "Name", "Dept", "Date", "Time", "Confidence"])
        self.report_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.report_table.setSelectionBehavior(QTableWidget.SelectRows)
        
        layout.addWidget(QLabel("📋 Attendance Records (All Dates)"))
        layout.addLayout(control_layout)
        layout.addWidget(self.report_table)
        
        tab.setLayout(layout)
        self.tabs.addTab(tab, "📊 Reports")
        
        self.load_attendance_report()
    
    def load_attendance_report(self):
        records = db.get_all_attendance()
        self.report_table.setRowCount(len(records))
        
        for row, (rec_id, emp_id, name, dept, time, conf, date) in enumerate(records):
            self.report_table.setItem(row, 0, QTableWidgetItem(str(rec_id)))
            self.report_table.setItem(row, 1, QTableWidgetItem(emp_id))
            self.report_table.setItem(row, 2, QTableWidgetItem(name))
            self.report_table.setItem(row, 3, QTableWidgetItem(dept))
            self.report_table.setItem(row, 4, QTableWidgetItem(date))
            self.report_table.setItem(row, 5, QTableWidgetItem(time))
            self.report_table.setItem(row, 6, QTableWidgetItem(f"{conf*100:.1f}%"))
    
    def delete_attendance(self):
        selected = self.report_table.selectedItems()
        if not selected:
            QMessageBox.warning(self, "Warning", "Select a record!")
            return
        
        dialog = PasswordDialog(self)
        if dialog.exec_() == QDialog.Accepted:
            password = dialog.get_password()
            if not db.verify_password(password):
                QMessageBox.critical(self, "Error", "❌ Invalid password!")
                return
        else:
            return
        
        row = selected[0].row()
        record_id = int(self.report_table.item(row, 0).text())
        name = self.report_table.item(row, 2).text()
        date = self.report_table.item(row, 4).text()
        
        reply = QMessageBox.question(self, "Confirm Delete", 
            f"Delete attendance record?\n\n{name}\nDate: {date}",
            QMessageBox.Yes | QMessageBox.No)
        
        if reply == QMessageBox.Yes:
            db.delete_attendance(record_id)
            QMessageBox.information(self, "Success", "✅ Deleted!")
            self.load_attendance_report()
    
    # ========================================================================
    # ADMIN TAB
    # ========================================================================
    
    def create_admin_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()
        
        self.admin_lock_widget = QWidget()
        lock_layout = QVBoxLayout()
        
        lock_label = QLabel("🔒 Admin Panel - Authentication Required")
        lock_label.setStyleSheet("font-size: 18px; font-weight: bold; color: #333;")
        lock_label.setAlignment(Qt.AlignCenter)
        
        unlock_btn = QPushButton("🔓 Unlock Admin Panel")
        unlock_btn.setStyleSheet("font-size: 16px; padding: 20px;")
        unlock_btn.clicked.connect(self.unlock_admin_panel)
        
        lock_layout.addStretch()
        lock_layout.addWidget(lock_label)
        lock_layout.addWidget(unlock_btn)
        lock_layout.addStretch()
        
        self.admin_lock_widget.setLayout(lock_layout)
        
        self.admin_content_widget = QWidget()
        content_layout = QVBoxLayout()
        
        content_layout.addWidget(QLabel("👥 Registered Employees"))
        
        admin_control_layout = QHBoxLayout()
        
        refresh_emp_btn = QPushButton("🔄 Refresh List")
        refresh_emp_btn.clicked.connect(self.load_employees)
        
        self.delete_emp_btn = QPushButton("🗑️ Delete Selected Employee")
        self.delete_emp_btn.setStyleSheet("background-color: #f44336; color: white;")
        self.delete_emp_btn.clicked.connect(self.delete_employee)
        
        admin_control_layout.addWidget(refresh_emp_btn)
        admin_control_layout.addWidget(self.delete_emp_btn)
        admin_control_layout.addStretch()
        
        self.employee_table = QTableWidget()
        self.employee_table.setColumnCount(6)
        self.employee_table.setHorizontalHeaderLabels(["ID", "Name", "Dept", "Designation", "Reg Date", "Faces"])
        self.employee_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.employee_table.setSelectionBehavior(QTableWidget.SelectRows)
        
        content_layout.addLayout(admin_control_layout)
        content_layout.addWidget(self.employee_table)
        
        info_label = QLabel("⚠️ Warning: Deleting an employee will remove all their face data and attendance records!")
        info_label.setStyleSheet("color: #f44336; font-weight: bold; padding: 10px;")
        content_layout.addWidget(info_label)
        
        self.admin_content_widget.setLayout(content_layout)
        self.admin_content_widget.hide()
        
        layout.addWidget(self.admin_lock_widget)
        layout.addWidget(self.admin_content_widget)
        
        tab.setLayout(layout)
        self.tabs.addTab(tab, "🔒 Admin")
    
    def unlock_admin_panel(self):
        dialog = PasswordDialog(self)
        if dialog.exec_() == QDialog.Accepted:
            password = dialog.get_password()
            if db.verify_password(password):
                self.admin_lock_widget.hide()
                self.admin_content_widget.show()
                self.load_employees()
                QMessageBox.information(self, "Success", "✅ Admin panel unlocked!")
            else:
                QMessageBox.critical(self, "Error", "❌ Invalid password!\n\nDefault: admin123")
    
    def load_employees(self):
        employees = db.get_all_employees()
        self.employee_table.setRowCount(len(employees))
        
        for row, (emp_id, name, dept, desig, reg_date) in enumerate(employees):
            self.employee_table.setItem(row, 0, QTableWidgetItem(emp_id))
            self.employee_table.setItem(row, 1, QTableWidgetItem(name))
            self.employee_table.setItem(row, 2, QTableWidgetItem(dept))
            self.employee_table.setItem(row, 3, QTableWidgetItem(desig))
            self.employee_table.setItem(row, 4, QTableWidgetItem(reg_date))
            
            face_count = db.get_employee_face_count(emp_id)
            self.employee_table.setItem(row, 5, QTableWidgetItem(str(face_count)))
    
    def delete_employee(self):
        selected = self.employee_table.selectedItems()
        if not selected:
            QMessageBox.warning(self, "Warning", "Select an employee to delete!")
            return
        
        dialog = PasswordDialog(self)
        if dialog.exec_() == QDialog.Accepted:
            password = dialog.get_password()
            if not db.verify_password(password):
                QMessageBox.critical(self, "Error", "❌ Invalid password!")
                return
        else:
            return
        
        row = selected[0].row()
        emp_id = self.employee_table.item(row, 0).text()
        name = self.employee_table.item(row, 1).text()
        face_count = self.employee_table.item(row, 5).text()
        
        reply = QMessageBox.question(self, "⚠️ Confirm Delete Employee", 
            f"Are you sure you want to DELETE this employee?\n\n"
            f"Employee: {name}\n"
            f"ID: {emp_id}\n"
            f"Face Images: {face_count}\n\n"
            f"This will permanently delete:\n"
            f"• Employee record\n"
            f"• All {face_count} face embeddings\n"
            f"• All attendance records\n\n"
            f"This action CANNOT be undone!",
            QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        
        if reply == QMessageBox.Yes:
            if db.delete_employee(emp_id):
                QMessageBox.information(self, "Success", 
                    f"✅ Employee '{name}' deleted successfully!\n\n"
                    f"Removed:\n"
                    f"• Employee record\n"
                    f"• {face_count} face embeddings\n"
                    f"• All attendance records")
                self.load_employees()
            else:
                QMessageBox.critical(self, "Error", "❌ Failed to delete employee!")


app = QApplication(sys.argv)
window = BLWAttendanceSystem()
window.show()

print("="*80)
print("✅ COMPLETE WORKING SYSTEM LAUNCHED")
print("   ✅ Registration: Working")
print("   ✅ Attendance: Working")
print("   ✅ All features active")
print("="*80)
print("\n📌 Default admin password: admin123")
print("="*80)

sys.exit(app.exec_())


🖥️  INITIALIZING COMPLETE WORKING GUI
✅ GUI Initialized
✅ COMPLETE WORKING SYSTEM LAUNCHED
   ✅ Registration: Working
   ✅ Attendance: Working
   ✅ All features active

📌 Default admin password: admin123
